# Predicting Diabetes 

Per [the Mayo Clinic](https://www.mayoclinic.org/diseases-conditions/diabetes/diagnosis-treatment/drc-20371451#:~:text=A%20fasting%20blood%20sugar%20level,separate%20tests%2C%20you%20have%20diabetes.), the Glycated Hemoglobin (A1C) test indicates a person's average blood sugar level for the past two to three months. It measures the percentage of blood sugar attached to hemoglobin, the oxygen-carrying protein in red blood cells. This test is a blood test which does not require fasting.

The higher your blood sugar levels, the more hemoglobin you'll have with sugar attached. 
* A1C level of 6.5% or higher on two separate tests indicates that you have diabetes
* A1C between 5.7 and 6.4 % indicates prediabetes
* A1C below 5.7 is considered normal

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 0)
import numpy as np

# styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

## Data Source

### National Health and Nutrition Examination Survey (NHANES)

The [National Health and Nutrition Examination Survey (NHANES)](https://www.cdc.gov/nchs/nhanes/about_nhanes.htm) is a program of continuous studies designed to assess the health and nutritional status of adults and children in the United States. The survey examines a nationally representative sample of about 5,000 persons located across the country each year. The survey is unique in that it combines interviews and physical examinations. The NHANES interview includes demographic, socioeconomic, dietary, and health-related questions. The examination component consists of medical, dental, and physiological measurements, as well as laboratory tests administered by highly trained medical personnel.

NHANES is a major program of the National Center for Health Statistics (NCHS). NCHS is part of the Centers for Disease Control and Prevention (CDC) and has the responsibility for producing vital and health statistics for the Nation.

### Subset for Predicting Diabetes

While NHANES collects a wealth of demographic, laboratory, and medical data, this analysis and predictive model uses a subset comprised of:

* Demographics - collected by trained interviewers using Computer-Assisted Personal Interview (CAPI) system in either English or Spanish, sometimes with assistance from an interpreter. Individuals 16 years and older and emancipated minors were interviewed directly; a proxy provided information for survey participants who were under 16 and for participants who could not answer the questions themselves.

* Body Measures - measured by trained health technicians in the Mobile Examination Center (MEC). 

* Smoking Survey

* Physical Activity Survey

* Blood Pressure

* Total Cholesterol

* A1C

* Insulin


In [13]:
demographic = pd.read_sas('./data/NHANES2017-2018_demographic.xpt')
activity = pd.read_sas('./data/NHANES2017-2018_physical_activity.xpt')
smoking = pd.read_sas('./data/NHANES2017-2018_smoking.xpt')

a1c = pd.read_sas('./data/NHANES2017-2018_a1c.xpt')
bp1 = pd.read_sas('./data/NHANES2017-2018_blood_pressure.xpt')
bp2 = pd.read_sas('./data/NHANES2017-2018_blood_pressure_oscillometric.xpt')
chol = pd.read_sas('./data/NHANES2017-2018_total_cholesterol.xpt')
insulin = pd.read_sas('./data/NHANES2017-2018_insulin.xpt')
measures = pd.read_sas('./data/NHANES2017-2018_body_measures.xpt')


In [14]:
data = [demographic, a1c, bp1, bp2, measures, insulin, activity, chol, smoking]

In [15]:
for f in data:
    f.SEQN = f.SEQN.map(lambda x: int(x))
    f.set_index('SEQN', inplace=True)
    display(f.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254 entries, 93703 to 102956
Data columns (total 45 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SDDSRVYR  9254 non-null   float64
 1   RIDSTATR  9254 non-null   float64
 2   RIAGENDR  9254 non-null   float64
 3   RIDAGEYR  9254 non-null   float64
 4   RIDAGEMN  597 non-null    float64
 5   RIDRETH1  9254 non-null   float64
 6   RIDRETH3  9254 non-null   float64
 7   RIDEXMON  8704 non-null   float64
 8   RIDEXAGM  3433 non-null   float64
 9   DMQMILIZ  6004 non-null   float64
 10  DMQADFC   561 non-null    float64
 11  DMDBORN4  9254 non-null   float64
 12  DMDCITZN  9251 non-null   float64
 13  DMDYRSUS  1948 non-null   float64
 14  DMDEDUC3  2306 non-null   float64
 15  DMDEDUC2  5569 non-null   float64
 16  DMDMARTL  5569 non-null   float64
 17  RIDEXPRG  1110 non-null   float64
 18  SIALANG   9254 non-null   float64
 19  SIAPROXY  9254 non-null   float64
 20  SIAINTRP  9254 non-null 

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6401 entries, 93705 to 102956
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   LBXGH   6045 non-null   float64
dtypes: float64(1)
memory usage: 100.0 KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8704 entries, 93703 to 102956
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   PEASCCT1  444 non-null    float64
 1   BPXCHR    1539 non-null   float64
 2   BPAARM    6817 non-null   float64
 3   BPACSZ    6812 non-null   float64
 4   BPXPLS    6742 non-null   float64
 5   BPXPULS   8281 non-null   float64
 6   BPXPTY    6742 non-null   float64
 7   BPXML1    6731 non-null   float64
 8   BPXSY1    6302 non-null   float64
 9   BPXDI1    6302 non-null   float64
 10  BPAEN1    6302 non-null   float64
 11  BPXSY2    6563 non-null   float64
 12  BPXDI2    6563 non-null   float64
 13  BPAEN2    6563 non-null   float64
 14  BPXSY3    6538 non-null   float64
 15  BPXDI3    6538 non-null   float64
 16  BPAEN3    6538 non-null   float64
 17  BPXSY4    556 non-null    float64
 18  BPXDI4    556 non-null    float64
 19  BPAEN4    556 non-null    float64
dtypes: float64(20)
memory us

...

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7435 entries, 93705 to 102956
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   LBXTC    6738 non-null   float64
 1   LBDTCSI  6738 non-null   float64
dtypes: float64(2)
memory usage: 174.3 KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6724 entries, 93705 to 102956
Data columns (total 36 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SMQ020    5856 non-null   float64
 1   SMD030    2359 non-null   float64
 2   SMQ040    2359 non-null   float64
 3   SMQ050Q   1338 non-null   float64
 4   SMQ050U   1255 non-null   float64
 5   SMD057    1338 non-null   float64
 6   SMQ078    793 non-null    float64
 7   SMD641    1063 non-null   float64
 8   SMD650    1022 non-null   float64
 9   SMD093    1021 non-null   float64
 10  SMDUPCA   6724 non-null   object 
 11  SMD100BR  6724 non-null   object 
 12  SMD100FL  929 non-null    float64
 13  SMD100MN  929 non-null    float64
 14  SMD100LN  929 non-null    float64
 15  SMD100TR  695 non-null    float64
 16  SMD100NI  695 non-null    float64
 17  SMD100CO  695 non-null    float64
 18  SMQ621    821 non-null    float64
 19  SMD630    42 non-null     float64
 20  SMQ661    14 non-null   

None

#### Demographics ([source](https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.htm))

Comprised of individual, family, and household-level information. 

In [22]:
demographic.describe().round(1)

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
count,9254.0,9254.0,9254.0,9254.0,597.0,9254.0,9254.0,8704.0,3433.0,6004.0,561.0,9254.0,9251.0,1948.0,2306.0,5569.0,5569.0,1110.0,9254.0,9254.0,9254.0,8780.0,8780.0,8780.0,6684.0,6684.0,6684.0,4977.0,9254.0,9254.0,9254.0,9254.0,9254.0,9254.0,9254.0,8764.0,9063.0,4751.0,9254.0,9254.0,9254.0,9254.0,8763.0,8780.0,8023.0
mean,10.0,1.9,1.5,34.3,10.4,3.2,3.5,1.5,107.5,1.9,1.5,1.2,1.1,9.3,6.3,3.5,2.7,2.0,1.1,1.7,2.0,1.1,2.0,2.0,1.1,2.0,2.0,1.1,3.7,3.6,0.5,0.9,0.5,1.5,2.9,2.1,1.5,2.1,34670.7,34670.7,1.5,141.0,12.5,12.2,2.4
std,0.0,0.2,0.5,25.5,7.1,1.3,1.7,0.5,70.6,0.3,0.6,1.6,0.5,18.6,5.8,1.2,3.1,0.4,0.3,0.5,0.2,0.3,0.0,0.2,0.3,0.1,0.1,0.4,1.7,1.8,0.8,1.1,0.8,0.5,0.8,0.7,0.7,0.7,41356.7,43344.0,0.5,4.2,17.3,17.2,1.6
min,10.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2571.1,0.0,1.0,134.0,1.0,1.0,0.0
25%,10.0,2.0,1.0,11.0,4.0,3.0,3.0,1.0,43.0,2.0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,2.0,13074.4,12347.3,1.0,137.0,6.0,6.0,1.0
50%,10.0,2.0,2.0,31.0,10.0,3.0,3.0,2.0,106.0,2.0,1.0,1.0,1.0,6.0,6.0,4.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,4.0,4.0,0.0,0.0,0.0,1.0,3.0,2.0,1.0,2.0,21098.5,21059.9,2.0,141.0,8.0,8.0,1.9
75%,10.0,2.0,2.0,58.0,17.0,4.0,4.0,2.0,166.0,2.0,2.0,1.0,1.0,7.0,9.0,4.0,5.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,2.0,1.0,2.0,4.0,2.0,2.0,3.0,36923.3,37562.0,2.0,145.0,14.0,14.0,3.7
max,10.0,2.0,2.0,80.0,24.0,5.0,7.0,2.0,239.0,9.0,7.0,99.0,9.0,99.0,66.0,9.0,77.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,7.0,7.0,3.0,3.0,3.0,2.0,4.0,3.0,3.0,3.0,433085.0,419762.8,2.0,148.0,99.0,99.0,5.0


In [18]:
demographic.columns

Index(['SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN', 'RIDRETH1',
       'RIDRETH3', 'RIDEXMON', 'RIDEXAGM', 'DMQMILIZ', 'DMQADFC', 'DMDBORN4',
       'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG',
       'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY', 'FIAINTRP',
       'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA', 'DMDHHSIZ', 'DMDFMSIZ',
       'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGZ', 'DMDHREDZ',
       'DMDHRMAZ', 'DMDHSEDZ', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA',
       'INDHHIN2', 'INDFMIN2', 'INDFMPIR'],
      dtype='object')

In [27]:
keepcols_demographic = ['RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMQMILIZ', 'DMDBORN4', 'DMDCITZN',
                    'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG', 'INDHHIN2', 'INDFMIN2', 'INDFMPIR']

In [29]:
keep_demographic = demographic[keepcols_demographic]
keep_demographic.rename(mapper={'RIAGENDR': 'gender',
                                'RIDAGEYR': 'age',
                                'RIDRETH3': 'race',
                                'DMQMILIZ': 'veteran_status',
                                'DMDBORN4': 'country_of_birth',
                                'DMDCITZN': 'citizen_status',
                                'DMDEDUC2': 'education',
                                'DMDMARTL': 'marital_status',
                                'RIDEXPRG': 'pregnancy_status',
                                'INDHHIN2': 'annual_household_income',
                                'INDFMIN2': 'annual_family_income',
                                'INDFMPIR': 'income_poverty_ratio'}, 
                       axis=1, inplace=True)
keep_demographic

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,annual_household_income,annual_family_income,income_poverty_ratio
SEQN,,,,,,,,,,,,
93703,2.0,2.0,6.0,NaN,1.0,1.0,NaN,NaN,NaN,15.0,15.0,5.00
93704,1.0,2.0,3.0,NaN,1.0,1.0,NaN,NaN,NaN,15.0,15.0,5.00
93705,2.0,66.0,4.0,2.0,1.0,1.0,2.0,3.0,NaN,3.0,3.0,0.82
93706,1.0,18.0,6.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
93707,1.0,13.0,7.0,NaN,1.0,1.0,NaN,NaN,NaN,10.0,10.0,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...
102952,2.0,70.0,6.0,2.0,2.0,1.0,3.0,1.0,NaN,4.0,4.0,0.95
102953,1.0,42.0,1.0,2.0,2.0,2.0,3.0,4.0,NaN,12.0,12.0,NaN
102954,2.0,41.0,4.0,2.0,1.0,1.0,5.0,5.0,2.0,10.0,10.0,1.18


In [30]:
keep_demographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254 entries, 93703 to 102956
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gender                   9254 non-null   float64
 1   age                      9254 non-null   float64
 2   race                     9254 non-null   float64
 3   veteran_status           6004 non-null   float64
 4   country_of_birth         9254 non-null   float64
 5   citizen_status           9251 non-null   float64
 6   education                5569 non-null   float64
 7   marital_status           5569 non-null   float64
 8   pregnancy_status         1110 non-null   float64
 9   annual_household_income  8763 non-null   float64
 10  annual_family_income     8780 non-null   float64
 11  income_poverty_ratio     8023 non-null   float64
dtypes: float64(12)
memory usage: 939.9 KB


Notes:
* Age 
    * Individuals aged 80 and over are topcoded at 80. In NHANES 2017-2018, the weighted mean age for participants 80+ is 85
    * Individuals' ages are reported in months for those 24 months (2 yrs) and younger
* Income-Poverty Ratio - calculated by dividing family (or individual) income by the poverty guidelines specific to the survey year. The value was not computed if the respondent only reported income as < $20,000 or ≥ $20,000. If family income was reported as a more detailed category, the midpoint of the range was used to compute the ratio. Values at or above 5.00 were coded as 5.00 or more because of disclosure concerns. The values were not computed if the income data was missing.


#### Body Measures

In [32]:
measures

,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,BMXBMI,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXHIP,BMIHIP
SEQN,,,,,,,,,,,,,,,,,,,,
93703,1.0,13.7,3.0,89.6,NaN,NaN,NaN,88.6,NaN,17.5,NaN,NaN,18.0,NaN,16.2,NaN,48.2,NaN,NaN,NaN
93704,1.0,13.9,NaN,95.0,NaN,NaN,NaN,94.2,NaN,15.7,NaN,NaN,18.6,NaN,15.2,NaN,50.0,NaN,NaN,NaN
93705,1.0,79.5,NaN,NaN,NaN,NaN,NaN,158.3,NaN,31.7,37.0,NaN,36.0,NaN,32.0,NaN,101.8,NaN,110.0,NaN
93706,1.0,66.3,NaN,NaN,NaN,NaN,NaN,175.7,NaN,21.5,46.6,NaN,38.8,NaN,27.0,NaN,79.3,NaN,94.4,NaN
93707,1.0,45.4,NaN,NaN,NaN,NaN,NaN,158.4,NaN,18.1,38.1,NaN,33.8,NaN,21.5,NaN,64.1,NaN,83.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952,1.0,49.0,NaN,NaN,NaN,NaN,NaN,156.5,NaN,20.0,34.4,NaN,32.6,NaN,25.1,NaN,82.2,NaN,87.3,NaN
102953,1.0,97.4,NaN,NaN,NaN,NaN,NaN,164.9,NaN,35.8,38.2,NaN,36.6,NaN,40.6,NaN,114.8,NaN,112.8,NaN
102954,1.0,69.1,NaN,NaN,NaN,NaN,NaN,162.6,NaN,26.1,39.2,NaN,35.2,NaN,26.8,NaN,86.4,NaN,102.7,NaN


In [33]:
measures.columns

Index(['BMDSTATS', 'BMXWT', 'BMIWT', 'BMXRECUM', 'BMIRECUM', 'BMXHEAD',
       'BMIHEAD', 'BMXHT', 'BMIHT', 'BMXBMI', 'BMXLEG', 'BMILEG', 'BMXARML',
       'BMIARML', 'BMXARMC', 'BMIARMC', 'BMXWAIST', 'BMIWAIST', 'BMXHIP',
       'BMIHIP'],
      dtype='object')

In [34]:
keepcols_measures = ['BMXWT', 'BMXHEAD', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 
                     'BMXARMC', 'BMXWAIST', 'BMXHIP']

In [35]:
keep_measures = measures[keepcols_measures]
keep_measures.rename(mapper={'BMXWT': 'weight_kg', 
                             'BMXHEAD': 'head_circumference_cm',
                             'BMXHT': 'height_cm',
                             'BMXBMI': 'BMI',
                             'BMXLEG': 'leg_length_cm',
                             'BMXARML': 'arm_length_cm', 
                             'BMXARMC': 'arm_circumference_cm',
                             'BMXWAIST': 'waist_circumference_cm',
                             'BMXHIP': 'hip_circumference_cm'},
                    axis=1, inplace=True)
keep_measures

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,weight_kg,head_circumference_cm,height_cm,BMI,leg_length_cm,arm_length_cm,arm_circumference_cm,waist_circumference_cm,hip_circumference_cm
SEQN,,,,,,,,,
93703,13.7,NaN,88.6,17.5,NaN,18.0,16.2,48.2,NaN
93704,13.9,NaN,94.2,15.7,NaN,18.6,15.2,50.0,NaN
93705,79.5,NaN,158.3,31.7,37.0,36.0,32.0,101.8,110.0
93706,66.3,NaN,175.7,21.5,46.6,38.8,27.0,79.3,94.4
93707,45.4,NaN,158.4,18.1,38.1,33.8,21.5,64.1,83.0
...,...,...,...,...,...,...,...,...,...
102952,49.0,NaN,156.5,20.0,34.4,32.6,25.1,82.2,87.3
102953,97.4,NaN,164.9,35.8,38.2,36.6,40.6,114.8,112.8
102954,69.1,NaN,162.6,26.1,39.2,35.2,26.8,86.4,102.7


#### Blood Pressure

#### Lab - A1C

#### Lab - Cholesterol

#### Lab - Insulin

#### Survey - Physical Activity

#### Survey - Smoking

## Preprocessing

NOTES TO SELF:
* Focus on adults; remove ages below 18 from dataset
* Remove pregnant women from dataset as their measurements may be skewed

# divider

In [ ]:
merged = pd.merge(a1c, demo, how='left', on='SEQN')
merged

In [ ]:
merged.LBXGH.isna().sum()

In [ ]:
merged.LBXGH.dropna(inplace=True)

In [ ]:
merged.LBXGH.describe().round(1)

In [ ]:
def diagnose(a1c_value):
    diagnosis = None
    if a1c_value >= 6.5:
        diagnosis = 'Diabetic'
    if (a1c_value >= 5.7) & (a1c_value < 6.5):
        diagnosis = 'Prediabetic'
    if a1c_value < 5.7:
        diagnosis = 'Normal'
    return diagnosis

In [ ]:
diagnose(merged.LBXGH[50])

In [ ]:
merged['calc_diagnosis'] = merged.LBXGH.map(lambda x: diagnose(x))
merged

In [ ]:
merged.calc_diagnosis.value_counts()

In [ ]:
merged.SEQN = merged.SEQN.map(lambda x: int(x))
merged